# Counting the invalid DOI-to-DOI citations in the CSV

In [1]:
from csv import reader

f = reader(open('invalid_dois.csv'))
header = next(f)
input_list = [(doi1, doi2) for (doi1, doi2) in f]
print(input_list[:50])

[('10.14778/1920841.1920954', '10.5555/646836.708343'), ('10.5406/ethnomusicology.59.2.0202', '10.2307/20184517'), ('10.1161/01.cir.63.6.1391', '10.1161/circ.37.4.509'), ('10.1177/1179546820918903', '10.3748/wjg.v10.i5.707.'), ('10.1080/10410236.2020.1731937', '10.1070/10810730903528033'), ('10.1161/strokeaha.112.652065', '10.1161/str.24.7.8322400'), ('10.1177/1049732310393747', '10.1111/j.545-5300.2003.42208.x'), ('10.1155/2017/1491405', '10.3760/cma.j.issn.0366-6999.20131202'), ('10.1161/01.res.68.6.1549', '10.1161/res.35.2.159'), ('10.4018/978-1-5225-2650-6.ch006', '10.1002/per'), ('10.1145/2525314.2594229', '10.5555/1873601.1873616'), ('10.1007/s10619-020-07320-z', '10.3390/sym11070911www.mdpi.com/journal/symmetry'), ('10.1007/s11771-020-4410-2', '10.13745/j.esf.2016.02.011'), ('10.1161/01.cir.102.5.591', '10.1161/circ.85.3.1537115'), ('10.1007/s40617-018-00299-1', '10.1901/jaba.2012.45-657'), ('10.1074/jbc.m508416200', '10.1059/0003-4819-100-4-483'), ('10.1177/2054358119836124', '

In [2]:
print(len(input_list))

1223296


In [3]:
invalid_dois = list(zip(*input_list))[1]
print(invalid_dois[:50])

('10.5555/646836.708343', '10.2307/20184517', '10.1161/circ.37.4.509', '10.3748/wjg.v10.i5.707.', '10.1070/10810730903528033', '10.1161/str.24.7.8322400', '10.1111/j.545-5300.2003.42208.x', '10.3760/cma.j.issn.0366-6999.20131202', '10.1161/res.35.2.159', '10.1002/per', '10.5555/1873601.1873616', '10.3390/sym11070911www.mdpi.com/journal/symmetry', '10.13745/j.esf.2016.02.011', '10.1161/circ.85.3.1537115', '10.1901/jaba.2012.45-657', '10.1059/0003-4819-100-4-483', '10.1016/j.amepre.2015.07.017.', '10.1016/j.atmosenv.2008.0305', '10.1161/circ.66.1.7083497', '10.1080/1364253032000157166', '10.1161/str.23.11.1440702', '10.1186/s12933-015-0183-6.', '10.1037/0021–9010.93.3.602', '10.1177/0312896216656720.', '10.5555/2133036.2133123', '10.5555/2442626.2442634', '10.5555/1785594.1785635', '10.1161/res.59.2.2874900', '10.1002/smj', '10.1161/res.89.12.1216', '10.3390/brainsci7120164.', '10.5555/944919.944937', '10.1145/2838344.2856460', '10.1177/0890117116661982.', '10.1161/circ.39.1.48', '10.110

# Useful References


1.   Xu, S., Hao, L., An, X. et al. Types of DOI errors of cited references in Web of Science with a cleaning method. Scientometrics 120, 1427–1437 (2019). https://doi.org/10.1007/s11192-019-03162-4

